In [1]:
from webull import webull # for paper trading, import 'paper_webull'
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime

# # Load environment variables from .env file
load_dotenv()
webull_email = os.environ['webull_email']
webull_pass = os.environ['webull_pass']
webull_access_token = os.environ['access_token']
webull_uuid = os.environ['webull_uuid']

wb = webull()

creds = wb.login(webull_email, webull_pass)
wb.api_login(access_token = webull_access_token, uuid = webull_uuid)
orders = wb.get_history_orders(status = 'All', count = 10000)
print(len(orders))

##if there's an rzone key error or some issue logging in it's likely due to an updated access_token or did. To correct:
##1. open incognito browser, log-in to the webull account with network tab open 
##2. open the v2 request and find the data needed, access token will be in one of the json requests, look in the headers. 

2658


In [2]:
order_details = [] 

for order in orders:
    if order['status'] == 'Working':
        continue
    else:
        if order['orders'][0]['tickerType'] != 'OPTION':
            combo_id = order['orders'][0]['comboId']
            order_id = (order['orders'][0]['orderId'])
            symbol = (order['orders'][0]['ticker']['symbol'])
            action = (order['orders'][0]['action'])
    #         order_type = (order['orders'][0]['orderType'])
            try:
                price = order['orders'][0]['avgFilledPrice']
            except Exception:
                try:
                    price = order['orders'][0]['lmtPrice']
                except Exception:
                    price = order['orders'][0]['auxPrice']

            total_quantity = (order['orders'][0]['totalQuantity'])
            filledAmount = (order['orders'][0]['filledAmount'])
            status = (order['orders'][0]['statusCode'])
            update_time = (order['orders'][0]['updateTime'])
            try:
                review_ = order['review_notes']
            except Exception:
                review_ = None
            order_type = 'EQUITY'
    #         strike_price = None
    #         type_ = None
            order_details.append([combo_id, order_id, symbol, action, order_type, price, total_quantity, filledAmount, status, update_time, review_])


        elif order['orders'][0]['tickerType'] == 'OPTION':
            combo_id = (order['orders'][0]['comboId'])
            order_id = (order['orders'][0]['orderId'])
            symbol = (order['orders'][0]['ticker']['symbol'])
            action = (order['orders'][0]['action'])
    #         order_type = (order['orders'][0]['orderType'])
            if order['status'] != 'Cancelled':
                price = (order['orders'][0]['avgFilledPrice'])
                total_quantity = (order['orders'][0]['totalQuantity'])
                filledAmount = (order['orders'][0]['filledAmount'])
            else:
                price = None
                total_quantity = None
                filledAmount = None
            status = (order['orders'][0]['statusCode'])
            update_time = (order['orders'][0]['updateTime'])
            review_ = None
            strike_price = order['orders'][0]['optionExercisePrice']
            type_ = order['orders'][0]['optionType'].upper()
            order_type = 'OPTION - '+str(strike_price)+' '+type_
            order_details.append([combo_id, order_id, symbol, action, order_type, price, total_quantity, filledAmount, status, update_time, review_])

df = pd.DataFrame(order_details)

df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,9QJ4E1OO782J9OHQSM6SHU4K0A,H6GQFBKP1CQ457SJDUPB00COTA,SPX,BUY,OPTION - 5425.00 CALL,6.00,1,600,Filled,06/13/2024 13:45:22 EDT,None
1,9QJ4E1OO782J9OHQSM6SHU4K0A,T5JEL1ETKM4F32P521NKVEEHKB,SPX,SELL,OPTION - 5425.00 CALL,None,None,None,Cancelled,06/13/2024 13:46:35 EDT,None
2,9QJ4E1OO782J9OHQSM6SHU4K0A,4NIS3LSCDL3NAR6C17N4KO6VI8,SPX,SELL,OPTION - 5425.00 CALL,5.10,1,510,Filled,06/13/2024 13:46:35 EDT,None
3,D2J2JGFHN0QJ5LBKB849QIKLDB,LIGUNU2L68C72HPGRG32TNO93B,SMCI,BUY,OPTION - 800.00 CALL,16.10,1,1610,Filled,06/13/2024 09:37:28 EDT,None
4,D2J2JGFHN0QJ5LBKB849QIKLDB,LUJS35O5KH9DAIF5SRHQ87M55A,SMCI,SELL,OPTION - 800.00 CALL,14.53,1,1453,Filled,06/13/2024 09:37:47 EDT,None


In [3]:
import pandas as pd

# df = pd.DataFrame(order_details)
df.columns = ['group_id','order_id','ticker','position_type','order_type',
              'price','quantity','cost','status','timestamp','review_notes'
#               'strike_price','option_type'
             ]
# df.head()


date_string = "06/01/2023 15:59:12 EDT"
dt = datetime.strptime(date_string, "%m/%d/%Y %H:%M:%S %Z")

date = dt.date()
day_of_week = dt.strftime("%A")
month = dt.strftime("%B")


# Define a lambda function to extract date, day of the week, and month
get_date_info = lambda x: pd.Series({
    "datetime" : datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z"),
    "date": datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z").date(),
    "day_of_week": datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z").strftime("%A"),
    "month": datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z").strftime("%m/01/%Y")
})

# Apply the lambda function to the 'datetime' column and assign the results to new columns
df[['datetime','date', 'day_of_week', 'month']] = df['timestamp'].apply(get_date_info)

# df

filled_orders = df[df['status'] == 'Filled'].sort_values(by = 'datetime', ascending = False).reset_index(drop = True)


for index,row in filled_orders.iterrows():
    if row['position_type'] in ['SHORT','SELL']:
#         break
        filled_orders.at[index,'quantity'] = float(row['quantity'])*-1
        
# filled_orders.head()

In [4]:
import uuid 

ids_ = []
filled_orders['order_group_id'] = None

for index,row in filled_orders.iterrows():
    if index+1 == len(filled_orders):
        break
    else:
        
        if len(filled_orders[filled_orders['group_id'] == row['group_id']]) > 1:
            filled_orders.at[index,'order_group_id'] = row['group_id']
        elif (filled_orders.loc[index+1]['ticker'] == row['ticker']) and (float(filled_orders.loc[index+1]['quantity']) + float(row['quantity']) == 0) and (type(row['order_group_id']) == float):
            id_ = str(uuid.uuid4())
            filled_orders.at[index,'order_group_id'] = id_
            filled_orders.at[index+1,'order_group_id'] = id_ 
        else:
            continue

tickers = filled_orders['ticker'].unique()
master = pd.DataFrame()

for ticker in tickers:
    ticker_data = filled_orders[filled_orders['ticker'] == ticker].sort_values(by = 'datetime',ascending = False).reset_index(drop = True)
    ticker_data['order_group_id'] = 'None'
    for index,row in ticker_data.iterrows():
        row = ticker_data.loc[index]
        if index+1 == len(ticker_data):
            ticker_data.at[index,'order_group_id'] = ticker_data.iloc[index-1]['order_group_id']
        elif row['order_group_id'] != 'None':
            continue
        elif len(ticker_data[ticker_data['group_id'] == row['group_id']]) > 1:
            ticker_data.at[index, 'order_group_id'] = row['group_id']
        elif (ticker_data.loc[index + 1, 'ticker'] == row['ticker'] and (float(ticker_data.loc[index + 1, 'quantity']) + float(row['quantity'])) == 0):
            id_ = str(uuid.uuid4())
            ticker_data.at[index, 'order_group_id'] = id_
            ticker_data.at[index + 1, 'order_group_id'] = id_
        else:
            continue
            
    master = master.append(ticker_data)

master = master.sort_values(by = 'datetime',ascending = False).reset_index(drop=True)
master = master[['ticker','position_type','order_type','price','quantity',
                'cost','status','timestamp','review_notes','datetime','date','day_of_week',
                'month','order_group_id'
               ]]
# master

/var/folders/wj/l40br62935bf3nrt3x44n1kw0000gn/T/ipykernel_94215/3701184117.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master = master.append(ticker_data)
/var/folders/wj/l40br62935bf3nrt3x44n1kw0000gn/T/ipykernel_94215/3701184117.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master = master.append(ticker_data)
/var/folders/wj/l40br62935bf3nrt3x44n1kw0000gn/T/ipykernel_94215/3701184117.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master = master.append(ticker_data)
/var/folders/wj/l40br62935bf3nrt3x44n1kw0000gn/T/ipykernel_94215/3701184117.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master = master.appe

In [5]:
orders_ = master[['order_group_id','order_type','ticker']].drop_duplicates().reset_index(drop = True)


for index,row in orders_.iterrows(): 
    trade_details = master[(master['order_group_id'] == row['order_group_id']) & (master['ticker'] == row['ticker'])]
    position_type = trade_details.iloc[-1]['position_type']
    position_size = trade_details.iloc[-1]['quantity']
    entry_price = trade_details.iloc[-1]['price']
    exit_price = trade_details.iloc[0]['price']
    entry_time = trade_details.iloc[-1]['timestamp']
    exit_time = trade_details.iloc[0]['timestamp']
    o_type = trade_details.iloc[0]['order_type']
    try:
        review_notes = ''.join(trade_details.dropna(subset = ['review_notes'])['review_notes'].unique())
    except Exception:
        review_notes = None
        
    
    if position_type == 'SHORT':
#         print(True)
        profit_ = ((float(position_size)*-1)*(float(entry_price)))-((float(position_size)*-1)*(float(exit_price)))
    else:
        if 'OPTION' in o_type:
#             print(True)
            profit_ = ((float(exit_price) - float(entry_price))*float(position_size))*100
        else:
            profit_ = ((float(position_size))*(float(exit_price)))-((float(position_size))*(float(entry_price)))
        
        
    
    orders_.loc[index, ['position_type', 'quantity', 'entry_price', 'exit_price', 'entry_time', 'exit_time','profit','review_notes']] = [position_type, position_size, entry_price, exit_price, entry_time, exit_time, profit_, review_notes]
    
# Define a lambda function to extract date, day of the week, and month
get_date_info = lambda x: pd.Series({
    "datetime" : datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z"),
    "date": datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z").date(),
    "day_of_week": datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z").strftime("%A"),
    "month": datetime.strptime(x, "%m/%d/%Y %H:%M:%S %Z").strftime("%m/01/%Y")
})

# Apply the lambda function to the 'datetime' column and assign the results to new columns
orders_[['datetime','date', 'day_of_week', 'month']] = orders_['entry_time'].apply(get_date_info).sort_values(by = 'datetime',ascending = False).reset_index(drop = True)
orders_



# Remove time zone abbreviation from datetime strings
orders_['entry_time'] = orders_['entry_time'].str.replace(r'\s+\w+$', '', regex=True)
orders_['exit_time'] = orders_['exit_time'].str.replace(r'\s+\w+$', '', regex=True)

# Convert to datetime objects
orders_['entry_time'] = pd.to_datetime(orders_['entry_time'], format='%m/%d/%Y %H:%M:%S')
orders_['exit_time'] = pd.to_datetime(orders_['exit_time'], format='%m/%d/%Y %H:%M:%S')

# Calculate the time difference in minutes
orders_['time_difference_minutes'] = (orders_['exit_time'] - orders_['entry_time']).dt.total_seconds() / 60


orders_.head()

,order_group_id,order_type,ticker,position_type,quantity,entry_price,exit_price,entry_time,exit_time,profit,review_notes,datetime,date,day_of_week,month,time_difference_minutes
0,9QJ4E1OO782J9OHQSM6SHU4K0A,OPTION - 5425.00 CALL,SPX,BUY,1,6.00,5.10,2024-06-13 13:45:22,2024-06-13 13:46:35,-90.0,,2024-06-13 13:45:22,2024-06-13,Thursday,06/01/2024,1.216667
1,D2J2JGFHN0QJ5LBKB849QIKLDB,OPTION - 800.00 CALL,SMCI,BUY,1,16.10,14.53,2024-06-13 09:37:28,2024-06-13 09:37:47,-157.0,,2024-06-13 09:37:28,2024-06-13,Thursday,06/01/2024,0.316667
2,MGG8LQ4CQC9IDK2T56NORB5IDA,OPTION - 5395.00 PUT,SPX,BUY,1,2.40,1.90,2024-06-12 15:29:08,2024-06-12 15:29:37,-50.0,,2024-06-12 15:29:08,2024-06-12,Wednesday,06/01/2024,0.483333
3,G4GM3LUF1ABR5OQDLC2K34L329,OPTION - 5405.00 PUT,SPX,BUY,2,4.30,4.10,2024-06-12 15:28:30,2024-06-12 15:28:50,-40.0,,2024-06-12 15:28:30,2024-06-12,Wednesday,06/01/2024,0.333333
4,98JQV25RL2CLESBIC56RT3E1DA,OPTION - 800.00 CALL,SMCI,BUY,1,17.20,15.50,2024-06-12 10:04:37,2024-06-12 10:06:29,-170.0,,2024-06-12 10:04:37,2024-06-12,Wednesday,06/01/2024,1.866667
